In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.question_answering import load_qa_chain
from pprint import pprint

In [ ]:
os.environ["OPENAI_API_KEY"] = "<This has been intentionally removed. All you must do is set your OpenAI API key in the environment variable OPENAI_API_KEY.>"

In [3]:
embedding_model = OpenAIEmbeddings()

/home/axiom/anaconda3/envs/NLP6/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
vector_path = '../models/vector-store-claude'
db_file_name = 'nlp_stanford'

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp'
)   

In [5]:
retriever = vectordb.as_retriever()

In [6]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  
    temperature=0.8  
)

question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

/home/axiom/anaconda3/envs/NLP6/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
# Define the prompt
prompt_template = """
    You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

In [8]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)

In [10]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})

/home/axiom/anaconda3/envs/NLP6/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.    

    Context: Introduction 
Swaraj Bhanja is a highly skilled Cloud Engineer, AI Enthusiast, and Technologist with 
expertise spanning cloud computing, DevOps, full-stack development, machine learning, 
and AI infrastructure optimization. His journey has been deﬁned by deep technical 
expertise, an insatiable curiosity for problem-solving, and a relentless drive for efficiency.  
 
Personal Information 
Born on October 21, 1997, in Jamshedpur, Jharkhand, India, he hails from Kochi, Kerala, 
and is currently based in Bangkok, Thailand, where he works as an AI Engineer at AI Brain 
Lab while 

In [12]:
pprint(answer)

{'answer': 'I am Swaraj Bhanja, a Cloud Engineer, AI Enthusiast, and '
           'Technologist with expertise in cloud computing, DevOps, full-stack '
           'development, machine learning, and AI infrastructure optimization. '
           "I'm currently based in Bangkok, Thailand, working as an AI "
           "Engineer at AI Brain Lab while pursuing my Master's in Data "
           'Science and Artificial Intelligence.',
 'chat_history': [],
 'question': 'Who are you by the way?',
 'source_documents': [Document(page_content='Introduction \nSwaraj Bhanja is a highly skilled Cloud Engineer, AI Enthusiast, and Technologist with \nexpertise spanning cloud computing, DevOps, full-stack development, machine learning, \nand AI infrastructure optimization. His journey has been deﬁned by deep technical \nexpertise, an insatiable curiosity for problem-solving, and a relentless drive for efficiency.  \n \nPersonal Information \nBorn on October 21, 1997, in Jamshedpur, Jharkhand, India, he h

In [13]:
# Get the answer
final_answer = answer['answer']

In [14]:
print(final_answer)

I am Swaraj Bhanja, a Cloud Engineer, AI Enthusiast, and Technologist with expertise in cloud computing, DevOps, full-stack development, machine learning, and AI infrastructure optimization. I'm currently based in Bangkok, Thailand, working as an AI Engineer at AI Brain Lab while pursuing my Master's in Data Science and Artificial Intelligence.


In [15]:
pprint(answer['source_documents'][0].metadata['source'])

'./Swaraj.pdf'
